In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("white")
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 100)

## Carga del dataset

Utilizando spark se realizo una partición del 2.5% de los datos del dataset original.

In [4]:
df = pd.read_csv("dataF.csv", sep = ",")
del df["Unnamed: 0"]
df.head()

,ult_actual,categoria,cupo_total_tc,ingreso_final,saldo_no_rot_mdo,cant_oblig_tot_sf,ind,gasto_familiar,cuotas_pagar,ingreso_poc,genre,pending_arrears,have_tc,age_cat_X,age_cat_Y,age_cat_Z,occupation_ama de casa,occupation_comerciante,occupation_desempleado con ingresos,occupation_desempleado sin ingresos,occupation_empleado,occupation_estudiante,occupation_ganadero,occupation_independiente,occupation_jubilado,occupation_pensionado,occupation_profesional independiente,occupation_rentista de capital,occupation_socio empleado - socio,civil_status_divorciado,civil_status_soltero,civil_status_union libre,civil_status_viudo,academic_level_especializacion,academic_level_ninguno,academic_level_primario,academic_level_tecnologo,academic_level_universitario,house_kind_familiar,house_kind_propia
0,20190429,1,6000000.00000,2629942.30000,21423000.00000,10.00000,1006462.49500,84040.00000,0.00000,0.00000,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
1,20180628,1,6000000.00000,2660041.28000,21949000.00000,6.00000,1417026.83200,240906.00000,0.00000,0.00000,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
2,20190429,1,6000000.00000,2599610.30000,22289000.00000,9.00000,1196746.69500,376766.00000,0.00000,0.00000,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
3,20200618,2,6000000.00000,3312905.18000,20636000.00000,12.00000,512597.86700,273565.00000,0.00000,0.00000,1,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
4,20200618,2,6000000.00000,3312905.18000,20636000.00000,12.00000,512438.85700,150000.00000,0.00000,0.00000,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0


In [5]:
df.shape

(830857, 40)

In [6]:
X = df.drop(["gasto_familiar"], axis = 1)
Y = df["gasto_familiar"]

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNetCV

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=7)

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.linear_model import ElasticNetCV

ENRegr = ElasticNetCV(random_state = 12)
scores = cross_validate(ENRegr, X, Y,return_train_score = True, cv=5)

In [20]:
scores = cross_validate(ENRegr, X, Y,return_train_score = True, cv=5)
print(f"Entrenamiento: {np.mean(scores['train_score'])}")
print(f"Test: {np.mean(scores['test_score'])}")

Entrenamiento: 0.09101057299640614
Test: 0.09077536862478137


### Modelo más lógico

In [21]:
from sklearn.ensemble import GradientBoostingRegressor

params = {'n_estimators': 500, 'max_depth': 6,
          'loss': 'huber','alpha':0.95}

GBReg = GradientBoostingRegressor(**params)
scores = cross_validate(GBReg, X, Y,return_train_score = True, cv=5)
print(f"Entrenamiento: {np.mean(scores['train_score'])}")
print(f"Test: {np.mean(scores['test_score'])}")

Entrenamiento: 0.1719339595857818
Test: 0.11285283357887817


In [22]:
params = {'n_estimators': 100, 'max_depth': 5,
          'loss': 'huber','alpha':0.95}

GBReg = GradientBoostingRegressor(**params)
scores = cross_validate(GBReg, X, Y,return_train_score = True, cv=5)
print(f"Entrenamiento: {np.mean(scores['train_score'])}")
print(f"Test: {np.mean(scores['test_score'])}")

KeyboardInterrupt: 

In [29]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error
scoring = ["r2", "neg_mean_absolute_percentage_error"]

ENRegr = ElasticNetCV(random_state = 0)
scores = cross_validate(ENRegr, X, Y,return_train_score = True, cv=5, scoring = scoring)

In [27]:
scores

{'fit_time': array([6.8589735 , 6.89479113, 6.80034018, 6.81441522, 6.79046392]),
 'score_time': array([0.0541842 , 0.05448937, 0.05335426, 0.05678463, 0.05011272]),
 'test_r2': array([0.09305511, 0.08625537, 0.09445936, 0.08895777, 0.09114923]),
 'train_r2': array([0.09042108, 0.0921329 , 0.09007838, 0.09148087, 0.09093964]),
 'test_neg_mean_absolute_percentage_error': array([  -31.41951487,   -21.02192513, -1001.91958688,   -31.77233946,
           -3.26081976]),
 'train_neg_mean_absolute_percentage_error': array([-265.94298586, -273.27068889,  -21.92756043, -267.02641301,
        -272.6559851 ])}

In [30]:
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we